# Парсинг
![title](https://raw.githubusercontent.com/vyushmanov/skillfactory_rds/master/module_6/parsing_of_sites.jpg)

In [1]:
import re
import json
import time
import requests
import numpy as np
import pandas as pd
import pandas_profiling

from pprint import pprint
from bs4 import BeautifulSoup

HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Content-Length': '137',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; ',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202002.03.092255',
    'x-client-date': '1580933207763',
    'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
    'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
    'x-requested-with': 'fetch'
}

url = "https://auto.ru/-/ajax/desktop/listing/"

In [2]:
# сделаем тестовый вызов запроса для BMW модели M6 и выведем его на экран, чтобы по нему можно было сделать функцию для создания 23 необходимых столбцов

# но сначала установим главные параметры запроса
mark = 'BMW'  # марка
model = 'M5'  # модель
geo_radius = 200  # радиус вокруг москвы = 200 км
geo_id = [213]  # Москва
top_days = "3"  # кол-во дней от текущей даты назад за которые запрашиваем объявления 

param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":top_days,
                "geo_radius":geo_radius,
                "geo_id":geo_id,
                'page': 1
            }

response = requests.post(url, json=param, headers = HEADERS)
df_ = response.json()
for elem in df_['offers']:
    print(elem)
    break

{'availability': 'IN_STOCK', 'category': 'cars', 'color_hex': '0000CC', 'delivery_info': {}, 'description': 'Состояние идеальное!\nКуплен у официального дилера за наличные.\nНДС нет!\nМаксимально возможная комплектация!\nАвтомобиль собран правильно.\nЕсть все модные/дорогостоящие опции.\nИндивидуальный, заказной  цвет кузова, и салона.\nЕстественно Без Дтп!\nПройдены все т.о., и выполнены все сервисные акции.\nНачало гарантии 04/2020\nОкончание 04/2023\nСтоимость нового автомобиля со всеми возможными скидками составила 11,5 млн.р.\n\nP.S.\nАвтомобиль очень красивый, стильный, яркий, дарящий незабываемые эмоции, и по настоящему подойдёт для истинного ценителя м5.\nУзнаваемость, и восхищенные взгляды на улицах любого города вам будут обеспечены в изобилии.\nНа мой взгляд это лучшее предложение в РФ.\nТорг символический!\n\nМодель: JF01 M5 Limousine\nЦвет: 490 Индивидуальный цвет\nВн обшивка: ZBTQ BMW Individual полная кожаная\n \nВыбор: 01CA Выбор автомобиля важного для СОР\n  01MA Спорт

In [3]:
# посмотрим какие столбцы нам нужны 
test = pd.read_csv('test.csv')
print('Кол-во столбцов теста:', test.shape[1])
print('Названия столбцов теста:', list(test.columns))

# вывод всех столбцов датафрейма
pd.set_option('display.max_columns', None)
test.head(2)

Кол-во столбцов теста: 32
Названия столбцов теста: ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 'description', 'engineDisplacement', 'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [4]:
df_marks_and_models = pd.read_csv('all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)
print('Кол-во марок: ', df_marks_and_models.shape[0])
display(df_marks_and_models.head(2))

Кол-во марок:  262


,Models
Mark,
AC,"['378_GT', 'ACE', 'ACECA', 'COBRA']"
ACURA,"['CL', 'CSX', 'EL', 'ILX', 'INTEGRA', 'LEGEND'..."


In [5]:
# кол-во объявлений на сайте на 9.9.2020 
dict_of_num_cars_of_models ={'AC':2, 'Acura':94, 'Alfa Romeo':65, 'Alpina':10, 
                             'AMC':2, 'Ariel':1, 'Asia':1, 'Aston Martin':25, 
                             'Audi':4093, 'Austin Healey':1, 'Bajaj':3, 
                             'Bentley':104, 'BMW':7423, 'Borgward':1, 
                             'Brilliance':10, 'Buick':17, 'BYD':22, 
                             'Cadillac':321, 'Changan':100, 'Chery':593, 
                             'Chevrolet':3670, 'Chrysler':316, 'Citroen':1142, 
                             'Dacia':9, 'Daewoo':1024, 'Daihatsu':47, 
                             'Daimler':5, 'Datsun':212, 'Delage':1, 'Derways':2, 
                             'DKW':1, 'Dodge':413, 'DongFeng':25, 'Doninvest':2, 
                             'DS':3, 'DW Hower':8, 'Eagle':1, 'Excalibur':1, 
                             'FAW':66, 'Ferrari':29, 'Fiat':206, 'Ford':4812, 
                             'Foton':2, 'GAC':14, 'Geely':302, 'Genesis':54, 
                             'GMC':26, 'Great Wall':420, 'Hafei':6, 'Haima':12, 
                             'Haval':110, 'Hawtai':4, 'Heinkel':1, 'Honda':1179, 
                             'HuangHai':1, 'Hummer':70, 'Hyundai':5712, 
                             'Infiniti':1041, 'Innocenti':1, 'Iran Khodro':21, 
                             'Isuzu':43, 'JAC':8, 'Jaguar':308, 'Jeep':466, 'JMC':1, 
                             'Kia':5595, 'LADA (ВАЗ)':8935, 'Lamborghini':14, 
                             'Lancia':8, 'Land Rover':1616, 'Lexus':985, 
                             'Lifan':295, 'Lincoln':62, 'Lotus':1, 'Luxgen':9, 
                             'Mahindra':1, 'Maserati':54, 'Maybach':7, 
                             'Mazda':2311, 'McLaren':5, 'Mercedes-Benz':8005, 
                             'Mercury':16, 'Metrocab':2, 'MG':10, 'MINI':313, 
                             'Mitsubishi':3935, 'Mitsuoka':3, 'Nissan':5776, 
                             'Oldsmobile':10, 'Opel':3131, 'Packard':2, 
                             'Peugeot':1636, 'Plymouth':13, 'Pontiac':38, 
                             'Porsche':714, 'Proton':3, 'PUCH':1, 'Ravon':112, 
                             'Renault':2771, 'Rolls-Royce':55, 'Rover':69, 
                             'Saab':147, 'Saturn':9, 'Scion':9, 'SEAT':77, 
                             'Shanghai Maple':1, 'Skoda':2810, 'Smart':89, 
                             'SsangYong':718, 'Subaru':934, 'Suzuki':878, 
                             'Tatra':5, 'Tesla':43, 'Tianma':2, 'Toyota':4648, 
                             'Triumph':1, 'Vauxhall':1, 'Volkswagen':7109, 
                             'Volvo':2069, 'Vortex':91, 'Wartburg':1, 'Willys':2, 
                             'Zotye':15, 'ZX':4, 'ГАЗ':935, 'Гоночный болид':2, 
                             'ЗАЗ':231, 'ЗИЛ':2, 'ИЖ':129, 'ЛуАЗ':45, 
                             'Москвич':198, 'СМЗ':4, 'ТагАЗ':68, 'УАЗ':1107}

In [6]:
# уберем модели с кол-вом объявлений ниже 300 и повысим регистр индексов 
list_keys_to_delete = []
for mark in dict_of_num_cars_of_models.keys():
    if dict_of_num_cars_of_models[mark] < 300:
        list_keys_to_delete.append(mark)
    
for key in list_keys_to_delete: del dict_of_num_cars_of_models[key]

for mark in list(dict_of_num_cars_of_models.keys()):
    mark_up = mark.upper()
    dict_of_num_cars_of_models[mark_up]=dict_of_num_cars_of_models.pop(mark)
    
# Удалим отечественные модели
for key in ['LADA (ВАЗ)','ГАЗ','УАЗ']: del dict_of_num_cars_of_models[key]

print(*dict_of_num_cars_of_models, sep=',')

AUDI,BMW,CADILLAC,CHERY,CHEVROLET,CHRYSLER,CITROEN,DAEWOO,DODGE,FORD,GEELY,GREAT WALL,HONDA,HYUNDAI,INFINITI,JAGUAR,JEEP,KIA,LAND ROVER,LEXUS,MAZDA,MERCEDES-BENZ,MINI,MITSUBISHI,NISSAN,OPEL,PEUGEOT,PORSCHE,RENAULT,SKODA,SSANGYONG,SUBARU,SUZUKI,TOYOTA,VOLKSWAGEN,VOLVO


In [7]:
dict_of_num_cars_of_models['GREAT WALL']

420

In [8]:
# исправим несоответствия
dict_of_num_cars_of_models['GREAT_WALL']=dict_of_num_cars_of_models.pop('GREAT WALL')
dict_of_num_cars_of_models['LAND_ROVER']=dict_of_num_cars_of_models.pop('LAND ROVER')
dict_of_num_cars_of_models['MERCEDES']=dict_of_num_cars_of_models.pop('MERCEDES-BENZ')
dict_of_num_cars_of_models['SSANG_YONG']=dict_of_num_cars_of_models.pop('SSANGYONG')

In [9]:
# запишем список релевантных марок для парсинга
list_of_relevant_marks = list(dict_of_num_cars_of_models.keys())

In [10]:
sum_of_ads = sum(dict_of_num_cars_of_models.values())
print(f"примерное кол-во строк после парсинга должно быть := {sum_of_ads}")

примерное кол-во строк после парсинга должно быть := 89189


In [11]:
# строка для проверки содержания элемента запроса
elem['additional_info']['hidden']

def utils_clean_values_dict(d_dict):
    for elem in d_dict:
        d_dict[elem] = 0
    return

In [12]:
df_['offers']

[{'availability': 'IN_STOCK',
  'category': 'cars',
  'color_hex': '0000CC',
  'delivery_info': {},
  'description': 'Состояние идеальное!\nКуплен у официального дилера за наличные.\nНДС нет!\nМаксимально возможная комплектация!\nАвтомобиль собран правильно.\nЕсть все модные/дорогостоящие опции.\nИндивидуальный, заказной  цвет кузова, и салона.\nЕстественно Без Дтп!\nПройдены все т.о., и выполнены все сервисные акции.\nНачало гарантии 04/2020\nОкончание 04/2023\nСтоимость нового автомобиля со всеми возможными скидками составила 11,5 млн.р.\n\nP.S.\nАвтомобиль очень красивый, стильный, яркий, дарящий незабываемые эмоции, и по настоящему подойдёт для истинного ценителя м5.\nУзнаваемость, и восхищенные взгляды на улицах любого города вам будут обеспечены в изобилии.\nНа мой взгляд это лучшее предложение в РФ.\nТорг символический!\n\nМодель: JF01 M5 Limousine\nЦвет: 490 Индивидуальный цвет\nВн обшивка: ZBTQ BMW Individual полная кожаная\n \nВыбор: 01CA Выбор автомобиля важного для СОР\n  0

In [13]:
dict_columns  = {'bodyType':0, 'brand':0, 'car_url':0, 'color':0, 'complectation_dict':0,
              'description':0, 'engineDisplacement':0,               
              'enginePower':0, 'equipment_dict':0, 'fuelType':0, 'mileage':0, 
              'modelDate':0, 'model_info':0, 'model_name':0, 'name':0, 'numberOfDoors':0, 
              'parsing_unixtime':0, 'priceCurrency':0, 'productionDate':0, 
              'sell_id':0, 'super_gen':0, 'vehicleConfiguration':0, 'vehicleTransmission':0,
              'vendor':0, 'Владельцы':0, 'Владение':0, 'ПТС':0, 'Привод':0, 
              'Руль':0, 'Состояние':0, 'Таможня':0, 'hidden':0, 'price':0}

# создаем функцию для обработки 31 необходимых столбцов из объявления в словарь
def get_one_car_to_dict(d_dict, d_elem):
    utils_clean_values_dict(d_dict)
    
    try: d_dict['bodyType'] = d_elem['vehicle_info']['configuration']['human_name']
    except: d_dict['bodyType'] = None
    try: d_dict['brand'] = d_elem['vehicle_info']['mark_info']['code']
    except: d_dict['brand'] = None

    d_dict['car_url'] = None

    try: d_dict['color'] = d_elem['color_hex']
    except: d_dict['color'] = None
    try: d_dict['complectation_dict'] = d_elem['vehicle_info']['configuration']
    except: d_dict['complectation_dict'] = None
    try: d_dict['description'] = d_elem['description']
    except: d_dict['description'] = None
    try: d_dict['engineDisplacement'] = d_elem['vehicle_info']['tech_param']['human_name'].split()[0]
    except: d_dict['engineDisplacement'] = None
    try: d_dict['enginePower'] = d_elem['vehicle_info']['tech_param']['power']
    except: d_dict['enginePower'] = None
    try: d_dict['equipment_dict'] = d_elem['vehicle_info']['equipment']
    except: d_dict['equipment_dict'] = None
    try: d_dict['fuelType'] = d_elem['lk_summary'].split()[-1]
    except: d_dict['fuelType'] = None
    try: d_dict['mileage'] = d_elem['state']['mileage']
    except: d_dict['mileage'] = None
    try: d_dict['modelDate'] = d_elem['vehicle_info']['super_gen']['year_from']
    except: d_dict['modelDate'] = None
    try: d_dict['model_info'] = d_elem['vehicle_info']['model_info']
    except: d_dict['model_info'] = None
    try: d_dict['model_name'] = d_elem['vehicle_info']['model_info']['code']
    except: d_dict['model_name'] = None
    try: d_dict['name'] = d_elem['vehicle_info']['tech_param']['human_name']
    except: d_dict['name'] = None
    try: d_dict['numberOfDoors'] = d_elem['vehicle_info']['configuration']['doors_count']
    except: d_dict['numberOfDoors'] = None
    
    d_dict['parsing_unixtime'] = None

    try: d_dict['priceCurrency'] = d_elem['price_info']['currency']
    except: d_dict['priceCurrency'] = None
    try: d_dict['productionDate'] = d_elem['documents']['year']
    except: d_dict['productionDate'] = None
    
    d_dict['sell_id'] = None

    try: d_dict['super_gen'] = d_elem['vehicle_info']['super_gen']
    except: d_dict['super_gen'] = None
    try: d_dict['vehicleConfiguration'] = d_elem['vehicle_info']['configuration']['body_type']\
                             +' '+d_elem['vehicle_info']['tech_param']['transmission']\
                             +' '+d_elem['vehicle_info']['tech_param']['human_name'].split()[0]
    except: d_dict['vehicleConfiguration'] = None
    try: d_dict['vehicleTransmission'] = d_elem['vehicle_info']['tech_param']['transmission']
    except: d_dict['vehicleTransmission'] = None
    try: d_dict['vendor'] = d_elem['vehicle_info']['vendor']
    except: d_dict['vendor'] = None
    try: d_dict['Владельцы'] = d_elem['documents']['owners_number']
    except: d_dict['Владельцы'] = None
    try: d_dict['Владение'] = d_elem['documents']['purchase_date']
    except: d_dict['Владение'] = None
    try: d_dict['ПТС'] = d_elem['documents']['pts']
    except: d_dict['ПТС'] = None
    try: d_dict['Привод'] = d_elem['lk_summary'].split(', ')[-2]
    except: d_dict['Привод'] = None
    try: d_dict['Руль'] = d_elem['vehicle_info']['steering_wheel']
    except: d_dict['Руль'] = None
    
    d_dict['Состояние'] = None
                             
    try: d_dict['Таможня'] = d_elem['documents']['custom_cleared']
    except: d_dict['Таможня'] = None
    try: d_dict['hidden'] = d_elem['additional_info']['hidden']
    except: d_dict['hidden'] = None
    try: d_dict['price'] = d_elem['price_info']['RUR']
    except: d_dict['price'] = None

    return
    
# тест функции 
get_one_car_to_dict(dict_columns, elem)

In [14]:
# проверяем формат тестового запроса с сайта и теста с кагл
display(pd.DataFrame(dict_columns).head(2))
test[test['enginePower']=='560 N12'].head(2)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,hidden,price
id,Седан,BMW,None,0000CC,21073941,Состояние идеальное!\nКуплен у официального ди...,Competition,625,NaN,бензин,9500,2017,NaN,M5,Competition 4.4 AT (625 л.с.) 4WD,4,None,RUR,2020,None,21073904,SEDAN AUTOMATIC Competition,AUTOMATIC,EUROPEAN,1,None,ORIGINAL,полный,LEFT,None,True,False,8990000
body_type,Седан,BMW,None,0000CC,SEDAN,Состояние идеальное!\nКуплен у официального ди...,Competition,625,NaN,бензин,9500,2017,NaN,M5,Competition 4.4 AT (625 л.с.) 4WD,4,None,RUR,2020,None,NaN,SEDAN AUTOMATIC Competition,AUTOMATIC,EUROPEAN,1,None,ORIGINAL,полный,LEFT,None,True,False,8990000


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
2385,лифтбек,AUDI,https://auto.ru/cars/used/sale/audi/rs7/110038...,серый,NaN,Audi Rs7 VSperformance Stage3\nНовый усиленный...,4.0 LTR,560 N12,NaN,бензин,https://autoru.naydex.net/ys1kR7800/fbd964zfcB...,39900,2013,"{""code"":""RS7"",""name"":""RS 7"",""ru_name"":""РС7"",""m...",RS7,4.0 AT (560 л.с.) 4WD,5,1603121542,RUB,2013,1100381316,"{""id"":""20045126"",""displacement"":3993,""engine_t...",LIFTBACK AUTOMATIC 4.0,автоматическая,EUROPEAN,3 или более,2 года и 4 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен
2576,лифтбек,AUDI,https://auto.ru/cars/used/sale/audi/rs7/110125...,чёрный,NaN,Внимание! Только для клиентов AVILON Автомобил...,4.0 LTR,560 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://autoru.naydex.net/ys1kR7800/fbd964zfcB...,181055,2014,"{""code"":""RS7"",""name"":""RS 7"",""ru_name"":""РС7"",""m...",RS7,4.0 AT (560 л.с.) 4WD,5,1603122317,RUB,2015,1101251163,"{""id"":""20183797"",""displacement"":3993,""engine_t...",LIFTBACK AUTOMATIC 4.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


# Parsing

In [15]:

auto_ru = {}

iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '
start_time_global = time.process_time()

for mark in list_of_relevant_marks:
    start_time = time.process_time()
    list_of_models = json.loads(df_marks_and_models.loc[mark, 'Models'].replace("'", '"'))
    for model in list_of_models:
        for num_page in range(1, 200):
            param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"900",
                "geo_radius":200,
                "geo_id":[213],
                'page': num_page
            }
            response = requests.post(url, json=param, headers = HEADERS)
            data = response.json()

            if data['offers'] != []:
                for elem in data['offers']:
                    get_one_car_to_dict(dict_columns, elem)
                    auto_ru[iter_of_ads] = dict_columns.copy()
                    iter_of_ads += 1
            else:
                break
    iter_of_marks +=1
    temp_1 = iter_of_ads-old_iter_of_ads
    temp_2 = dict_of_num_cars_of_models[mark]
    temp_3 = temp_1/temp_2
    temp_4 = space*abs(13-len(mark)-len(str(iter_of_marks)))
    temp_5 = int(time.process_time() - start_time)
    temp_8 = round(temp_3*100,2)
    temp_7 = space*abs(18-len(str(temp_1))-len(str(temp_2))-len(str(temp_8))-len(str(temp_5)))
    temp_9 = int(time.process_time() - start_time_global)
    

    sum_ads += temp_1
    sum_ads_from_dict += temp_2
    temp_def = sum_ads/sum_ads_from_dict
    temp_6 = (sum_of_ads*temp_def - iter_of_ads)*temp_9/sum_ads


    print (f'''{iter_of_marks}.{mark} {temp_4} ушло:{temp_5} сек., стр.{temp_1} из {temp_2} {temp_8}%,{temp_7}всего стр. {iter_of_ads}, прогноз {int(temp_6/60)} мин.''')

    old_iter_of_ads=iter_of_ads

print (f'''Всего спарсили := {iter_of_ads} строк, на это ушло:{int((time.process_time() - start_time_global)/60)} мин., ''')
       
# запись датасета
df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv('all_auto_ru_2021-01-08.csv', index=False)
print(f'кол-во строк датасета:= {len(df_auto_ru)}')
       
# визуальный контроль
df_auto_ru.head(3)       

1.AUDI          ушло:4 сек., стр.3289 из 4093 80.36%,    всего стр. 3289, прогноз 1 мин.
2.BMW           ушло:7 сек., стр.5944 из 7423 80.08%,    всего стр. 9233, прогноз 1 мин.
3.CADILLAC      ушло:1 сек., стр.366 из 321 114.02%,     всего стр. 9599, прогноз 1 мин.
4.CHERY         ушло:1 сек., стр.389 из 593 65.6%,       всего стр. 9988, прогноз 1 мин.
5.CHEVROLET     ушло:4 сек., стр.2299 из 3670 62.64%,    всего стр. 12287, прогноз 1 мин.
6.CHRYSLER      ушло:1 сек., стр.209 из 316 66.14%,      всего стр. 12496, прогноз 1 мин.
7.CITROEN       ушло:2 сек., стр.885 из 1142 77.5%,      всего стр. 13381, прогноз 1 мин.
8.DAEWOO        ушло:1 сек., стр.572 из 1024 55.86%,     всего стр. 13953, прогноз 1 мин.
9.DODGE         ушло:1 сек., стр.280 из 413 67.8%,       всего стр. 14233, прогноз 1 мин.
10.FORD         ушло:4 сек., стр.2933 из 4812 60.95%,    всего стр. 17166, прогноз 1 мин.
11.GEELY        ушло:0 сек., стр.238 из 302 78.81%,      всего стр. 17404, прогноз 1 мин.
12.HONDA      

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,hidden,price
0,Седан,AUDI,None,EE1D19,"{'id': '7879466', 'body_type': 'SEDAN', 'doors...",11.11.2020 перебрана вся топливная \n\nПокупал...,2.3,133,{},бензин,356000,1990,"{'code': '100', 'name': '100', 'ru_name': '100...",100,2.3 MT (133 л.с.),4,None,RUR,1992,None,"{'id': '7879464', 'name': 'IV (C4)', 'ru_name'...",SEDAN MECHANICAL 2.3,MECHANICAL,EUROPEAN,3,None,ORIGINAL,передний,LEFT,None,True,False,120000
1,Седан,AUDI,None,0000CC,"{'id': '7879466', 'body_type': 'SEDAN', 'doors...",ЛОТ: 01237660\nFAVORIT MOTORS KIA Север\n\nВы ...,2.3,133,"{'alloy-wheel-disks': True, 'tinted-glass': Tr...",бензин,786115,1990,"{'code': '100', 'name': '100', 'ru_name': '100...",100,2.3 MT (133 л.с.),4,None,RUR,1993,None,"{'id': '7879464', 'name': 'IV (C4)', 'ru_name'...",SEDAN MECHANICAL 2.3,MECHANICAL,EUROPEAN,3,None,ORIGINAL,передний,LEFT,None,True,False,125000
2,Седан,AUDI,None,0000CC,"{'id': '7879466', 'body_type': 'SEDAN', 'doors...",Тойота Центр Кунцево – официальный дилер!\nУ н...,2.3,133,{},бензин,250000,1990,"{'code': '100', 'name': '100', 'ru_name': '100...",100,2.3 MT (133 л.с.),4,None,RUR,1991,None,"{'id': '7879464', 'name': 'IV (C4)', 'ru_name'...",SEDAN MECHANICAL 2.3,MECHANICAL,EUROPEAN,3,None,DUPLICATE,передний,LEFT,None,True,False,138000


In [16]:
# запись датасета
df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv('all_auto_ru_2021-01-08.csv', index=False)
print(f'кол-во строк датасета:= {len(df_auto_ru)}')


кол-во строк датасета:= 73440


In [17]:
pd.read_csv('all_auto_ru_2021-01-08.csv')

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,hidden,price
0,Седан,AUDI,NaN,EE1D19,"{'id': '7879466', 'body_type': 'SEDAN', 'doors...",11.11.2020 перебрана вся топливная \n\nПокупал...,2.3,133.0,{},бензин,356000,1990.0,"{'code': '100', 'name': '100', 'ru_name': '100...",100,2.3 MT (133 л.с.),4.0,NaN,RUR,1992,NaN,"{'id': '7879464', 'name': 'IV (C4)', 'ru_name'...",SEDAN MECHANICAL 2.3,MECHANICAL,EUROPEAN,3.0,NaN,ORIGINAL,передний,LEFT,NaN,True,False,120000.0
1,Седан,AUDI,NaN,0000CC,"{'id': '7879466', 'body_type': 'SEDAN', 'doors...",ЛОТ: 01237660\nFAVORIT MOTORS KIA Север\n\nВы ...,2.3,133.0,"{'alloy-wheel-disks': True, 'tinted-glass': Tr...",бензин,786115,1990.0,"{'code': '100', 'name': '100', 'ru_name': '100...",100,2.3 MT (133 л.с.),4.0,NaN,RUR,1993,NaN,"{'id': '7879464', 'name': 'IV (C4)', 'ru_name'...",SEDAN MECHANICAL 2.3,MECHANICAL,EUROPEAN,3.0,NaN,ORIGINAL,передний,LEFT,NaN,True,False,125000.0
2,Седан,AUDI,NaN,0000CC,"{'id': '7879466', 'body_type': 'SEDAN', 'doors...",Тойота Центр Кунцево – официальный дилер!\nУ н...,2.3,133.0,{},бензин,250000,1990.0,"{'code': '100', 'name': '100', 'ru_name': '100...",100,2.3 MT (133 л.с.),4.0,NaN,RUR,1991,NaN,"{'id': '7879464', 'name': 'IV (C4)', 'ru_name'...",SEDAN MECHANICAL 2.3,MECHANICAL,EUROPEAN,3.0,NaN,DUPLICATE,передний,LEFT,NaN,True,False,138000.0
3,Седан,AUDI,NaN,CACECB,"{'id': '7879466', 'body_type': 'SEDAN', 'doors...",Авто в хорошем состоянии стоит новая зимняя р...,2.3,133.0,{},бензин,300000,1990.0,"{'code': '100', 'name': '100', 'ru_name': '100...",100,2.3 MT (133 л.с.),4.0,NaN,RUR,1991,NaN,"{'id': '7879464', 'name': 'IV (C4)', 'ru_name'...",SEDAN MECHANICAL 2.3,MECHANICAL,EUROPEAN,3.0,NaN,ORIGINAL,передний,LEFT,NaN,True,False,130000.0
4,Седан,AUDI,NaN,FAFBFB,"{'id': '7879466', 'body_type': 'SEDAN', 'doors...",Предлагаю вашему вниманию; легендарный автомоб...,2.6,150.0,"{'engine-proof': True, 'alloy-wheel-disks': Tr...",бензин,337000,1990.0,"{'code': '100', 'name': '100', 'ru_name': '100...",100,2.6 MT (150 л.с.),4.0,NaN,RUR,1994,NaN,"{'id': '7879464', 'name': 'IV (C4)', 'ru_name'...",SEDAN MECHANICAL 2.6,MECHANICAL,EUROPEAN,3.0,"{'year': 2018, 'month': 3}",DUPLICATE,передний,LEFT,NaN,True,False,175000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73435,Минивэн,SSANG_YONG,NaN,4A2197,"{'id': '10408064', 'body_type': 'MINIVAN', 'do...",Код предложения 1264820-ФЛ/МТ2-15\nКод предлож...,2.0d,149.0,"{'cruise-control': True, 'esp': True, 'airbag-...",дизель,133357,2013.0,"{'code': 'STAVIC', 'name': 'Stavic', 'ru_name'...",STAVIC,2.0d AT (149 л.с.) 4WD,5.0,NaN,RUR,2014,NaN,"{'id': '10408061', 'year_from': 2013, 'price_s...",MINIVAN AUTOMATIC 2.0d,AUTOMATIC,KOREAN,1.0,"{'year': 2015, 'month': 6}",ORIGINAL,полный,LEFT,NaN,True,False,880000.0
73436,Минивэн,SSANG_YONG,NaN,040001,"{'id': '10408064', 'body_type': 'MINIVAN', 'do...",Год выпуска октябрь 2014г.\nКуплен в марте 201...,2.0d,149.0,"{'cruise-control': True, 'asr': True, 'tinted-...",дизель,83700,2013.0,"{'code': 'STAVIC', 'name': 'Stavic', 'ru_name'...",STAVIC,2.0d AT (149 л.с.) 4WD,5.0,NaN,RUR,2014,NaN,"{'id': '10408061', 'year_from': 2013, 'price_s...",MINIVAN AUTOMATIC 2.0d,AUTOMATIC,KOREAN,1.0,"{'year': 2015, 'month': 3}",ORIGINAL,полный,LEFT,NaN,True,False,1290000.0
73437,Минивэн,SSANG_YONG,NaN,040001,"{'id': '10408064', 'body_type': 'MINIVAN', 'do...",Сразу отвечу на вопросы «Почему так дорого» и ...,2.0d,149.0,"{'cruise-control': True, 'tinted-glass': True,...",дизель,215000,2013.0,"{'code': 'STAVIC', 'name': 'Stavic', 'ru_name'...",STAVIC,2.0d AT (149 л.с.) 4WD,5.0,NaN,RUR,2014,NaN,"{'id': '10408061', 'year_from': 2013, 'price_s...",MINIV